# Overdetermined least-squares problems

This notebook explores three ways to solve an overdetermined least-squares problem from approximation theory (taken from Lecture 19 in Trefethen and Bau). Suppose we want to fit a degree $14$ polynomial $p(x) = c_0 + c_1 x + \cdots + c_{14} x^{14}$ to $100$ data points $b_k = B\exp(\sin(4x_k))$ at equispaced points $x_1,x_2,\ldots,x_{100}$ in $[0,1]$, where $B$ is a normalization constant so that the least squares solution has $c_{14} = 1$. Each data point gives us an equation for the $15$ unkown polynomial coefficients: the result is the $100\times 15$ overdetermined _Vandermonde_ system $Ac=b$,

$$
\begin{pmatrix}
1 && x_1 && \cdots && x_1^{14} \\
1 && x_2 && \cdots && x_2^{14} \\
\ddots && \ddots &&\vdots && \ddots \\
1 && x_{100} && \cdots && x_{100}^{14}
\end{pmatrix}
\begin{pmatrix}
c_0 \\ c_1 \\ \ddots \\ c_{14}
\end{pmatrix}
=
\begin{pmatrix}
b_1 \\ b_2 \\ \ddots \\ b_{100}
\end{pmatrix}
$$

The system has more equations than unknowns and there is no unique solution. The Vandermonde matrix has full column rank but is highly ill-conditioned.

In [ ]:
using LinearAlgebra

In [ ]:
# set up least-squares problem A * c = b so that a[15] = 1
m = 100
n = 15
xpts = collect(0:m-1)/(m-1)
A = zeros(m,n)
for k in 1:n
    A[:,k] = xpts .^ (k-1)
end
b = exp.(sin.(4*xpts)) / 2006.787453080206

cond(A)             # display condition number of Vandermonde matrix

Before we test out the three methods to solve $\min\|Ac-b\|$, let's estimate the condition number of the least-squares solution $c_*$ associated with perturbations in the right-hand side, $\kappa_{b\rightarrow c}$, and the coefficient matrix, $\kappa_{A\rightarrow c}$. We'll use "backslash" to get the approximate least-squares solution.

In [ ]:
c = A \ b
y = A * c
κ = cond(A)
cosθ = norm(y)/norm(b)
η = norm(A)*norm(c) / norm(y)
κA = κ / cosθ
κb = κ / (η * cosθ)
display(κA)                     # display condition number for perturbations to A
display(κb)                     # display condition number for perturbations to b

Now, let's compare the three methods to compute the least-squares solution. The least-squares solution has $a_{14} = 1$ by construction.  


## 1. The Normal Equations
 
This means solving $A^TAc = A^Tb$.

In [ ]:
# normal equations
cN = (A'*A) \ (A' * b)
abs(cN[end]-1)

Not even a single correct digit! Computing the least-squares solution via the normal equations is unstable when the system is ill-conditioned. It is stable for restricted classes of problems, e.g., problems with bounded condition number.

## 2. The Singular Value Decomposition

Now, let's try diagonal reduction with the thin SVD, $A = U\Sigma V^T$. We need to solve $\Sigma d = U^T b$ and then $c = Vd$.

In [ ]:
# diagonal reduction (SVD)
F = svd(A)
d = (F.U' * b) ./ F.S
c = F.V * d
abs(1-c[end])

Seven correct digits is much better! This is about as many correct digits as we can hope for when $\kappa_{A\rightarrow c}\approx 10^{10}$. The SVD approach is backward stable.

## The QR Decomposition (three ways)

Finally, let's compute the least-squares solution using the thin QR factorization, $A=QR$. We need to solve $Rc = Q^Tb$.

First, we will compute $A = QR$ with modified Gram-Schmidt.

In [ ]:
function mgs(A)
    V = copy(A)
    n = size(A,2)               # number of columns
    R = UpperTriangular(zeros(n,n))
    for i in 1:n
        R[i,i] = norm(V[:,i])
        V[:,i] = V[:, i] / R[i,i]
        for j in i+1:n
            R[i,j] = V[:,i]'*V[:,j]
            V[:,j] = V[:,j] - R[i,j]*V[:,i]
        end
    end

    return V, R
end

# unpack Q and R from modified Gram-Schmidt
F = mgs(A)
Q = F[1]            # 100 x 15 ONB from Gram-Schmidt
R = F[2]            # 15 x 15 upper triangular matrix from Gram-Schmidt

# solve R * c = Q' * b
c = R \ (Q' * b)
abs(1-c[end])

Modified Gram-Schmidt is also performing pretty poorly here! A part of this is the loss of orthogonality in the columns of $Q$ due to ill-conditioning in $A$, exacerbated by ill-conditioning in $R$ (which has the same condition number as $A$). However, we can stabilize modified Gram-Schmidt by computing the product $Q^Tb$ in a clever way. We orthogonalize the augmented matrix $[A\,\, b]$ with MGS and extract the last column, which is (in exact arithmetic) $Q^T b$.

In [ ]:
F = mgs([A b])

# unpack Q and R from modified Gram-Schmidt
R = F[2]            # 16 x 16 upper triangular matrix from Gram-Schmidt
Qb = R[1:n,n+1]     # extract Q'*b from the last column of ONB for augmented matrix
R = R[1:n,1:n]      # extract R from principle n x n submatrix of triangular factor

# solve R * c = Q' * b
c = R \ Qb
abs(1-c[end])

Seven digits again! We are doing almost as well as the SVD! Remarkably, this augmented MGS method is a backward stable method for computing least-squares solutions, even though MGS is not a backward stable algorithm for factoring $A=QR$.

Finally, let's try out the gold standard algorithm for "daily-use" - Householder QR.

In [ ]:
F = qr(A)
Qb = F.Q' * b
c = F.R \ Qb[1:n]
abs(1-c[end])

Householder QR gives us about 6 digits of accuracy, which is (again) about as many correct digits as we can hope for from a least-squares problem with condition number $\kappa_{A\rightarrow c} \approx 10^{10}$. 

## Beyond numerical linear algebra

The ill-conditioning in this polynomial regression problem can be avoided altogether by working with 

* better polynomial bases than the monomials $1, x, \ldots, x^{14}$, and
* better sample points that equispaced points in the unit interval $[-1, 1]$.

The first $15$ Chebyshev polynomials are a much better choice of basis. We can build the matrix column by column using the three term recurrence satisfied by the Chebyshev polynomials, with $T_0(x) = 1$, $T_1(x) = x$, and

$$ T_{k+1}(x) = 2xT_k(x) - T_{k-1}(x).$$

The roots of the Chebyshev polynomials are an excellent set of sample points to build approximations from:

$$x_k = \cos(\pi (k + 1/2)/100), \qquad k = 1,2,\ldots,100.$$

What is the condition number of the Vandermonde matrix associated with Chebyshev polynomials and points?





In [ ]:
# set up a least-squares problem A * c = b so that a[15] = 1, in Chebyshev basis
m = 100
n = 15
xpts = cos.(pi*(0.5.+collect(0:m-1))/m)
A = zeros(m,n)
A[:,1] = ones(m,1)
A[:,2] = xpts
for k in 3:n
    A[:,k] = 2*xpts.*A[:,k-1] - A[:,k-2] # xpts .^ (k-1) #
end
b = exp.(sin.(4*xpts)) / 2006.787453080206

# condition number of Vandermonde matrix
cond(A)

This is really the domain of _approximation theory_: how can we represent and construct highly accurate approximations to continuous objects like functions on a computer? What bases should we employ? Where should we sample/measure our function? The now classic reference is Approximation Theory and Practice by Nick Trefethen (who happens to be the first author of our NLA textbook).

If you want to check accuracy in the solution, note that we have changed the least-squares problem entirely. The least-squares solution to _this problem_ may not be normalized to $1$ at the endpoint anymore. However, we can check the condition numbers again and be confident that a backward-stable algorithm achieves about 15 digits of accuracy.

In [ ]:
c = A \ b
y = A * c
κ = cond(A)
cosθ = norm(y)/norm(b)
η = norm(A)*norm(c) / norm(y)
κA = κ / cosθ
κb = κ / (η * cosθ)
display(κA)
display(κb) 